# Embedding of the Bible Interpreted

### Setting the working directory, loading libraries and data

In [1]:
# Importing library
import json
import pandas as pd
import os

# Function to set the wd as the root of the repository
def find_repo_root(repo_name):
    current_dir = os.getcwd()
    while current_dir != '/':
        if os.path.basename(current_dir) == repo_name:
            return current_dir
        current_dir = os.path.dirname(current_dir)
    raise FileNotFoundError(f"Repository root '{repo_name}' not found.")

# Setting the working directory
repo_name = 'orare-model'
repo_root = find_repo_root(repo_name)
os.chdir(repo_root)

# Loading the data
with open('bible/data/bible_by_theme_int_v2.json') as f:
    data = json.load(f)

# Converting json into df
bible_data = pd.DataFrame(data)
bible_data.head()

,id,pasaje,texto,interpretación,temas
0,vec1,1 Corintios 10:12,"Así que, el que piensa estar firme, mire no ca...",Este pasaje nos recuerda la importancia de la ...,[orgullo]
1,vec2,1 Corintios 10:13,"No os ha tomado tentación, sino humana: mas fi...",Este pasaje nos recuerda que las tentaciones y...,"[crecimiento espiritual, maternidad, vicio]"
2,vec3,1 Corintios 10:31,"Si pues coméis, ó bebéis, ó hacéis otra cosa, ...",Este pasaje nos recuerda que todas nuestras ac...,[descanso y vacaciones]
3,vec4,1 Corintios 11:9,Porque tampoco el varón fué criado por causa d...,Este pasaje de la primera carta a los Corintio...,[creación/evolución]
4,vec5,1 Corintios 13:13,"Y ahora permanecen la fe, la esperanza, y la c...",Este pasaje subraya la importancia de tres vir...,[amor]


### Setting the connection to OpenAI API to do the embeddings

In [2]:
# Setting the libraries and connection
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# Loading api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Using OpenAI API to do the embeddings
client = openai.OpenAI()

### Applying the embeddings

In [3]:
# Function to apply the embeddings
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [4]:
# Applying the embeddings
bible_data['interpretacion_vector'] = bible_data.interpretación.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))

In [5]:
# Saving the embeddings in a file
bible_data.to_csv('bible/data/bible_by_theme_int_embedding_v2.txt',sep='|',index=False, encoding='utf-8')